In [30]:
import numpy as np
from scipy.linalg import sqrtm, logm
import mne

class GenerateurDonnees:
    def __init__(self, channels=3, sfreq=256.0, duration=10, change=5.0, A1=2.0e-6, A2=0.0e-6, sd=2.0e-6):
        self.channels = channels
        self.sfreq = sfreq
        self.duration = duration
        self.change = change
        self.frequencies = 10.0 + 2.0 * np.random.rand(channels)
        self.phases = 2.0 * np.pi * np.random.rand(channels)
        self.A1 = A1
        self.A2 = A2
        self.sd = sd
        self.info = mne.create_info([f'EEG{n:03}' for n in range(1, channels + 1)],
                                    ch_types=['eeg'] * channels, sfreq=sfreq)

    def generer_donnees(self):
        samples = int(self.sfreq * self.duration)
        t = np.linspace(0, self.duration, samples, endpoint=False)
        A = self.A1 + (t > self.change).astype(float) * (self.A2 - self.A1)
        data = A * np.cos(np.outer(self.frequencies, t) + np.outer(self.phases, np.ones(samples)))
        data += np.random.normal(0, self.sd, size=(self.channels, samples))
        # Vérification des valeurs infinies ou NaN
        if not np.isfinite(data).all():
            raise ValueError("Les données générées contiennent des valeurs infinies ou NaN")
        raw = mne.io.RawArray(data, self.info)
        return raw

    def exporter_donnees(self, filename='dummy.edf'):
        raw = self.generer_donnees()
        mne.export.export_raw(filename, raw, fmt='edf', overwrite=True)

class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            #covariance_matrix = np.cov(segment) + np.eye(k) * 1e-6  # Régularisation ajoutée
            covariance_matrix = np.cov(segment)
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

class Transformation:
    def __init__(self):
        pass

    def project_to_tangent(self, cov_matrix_current, cov_matrix_next):
        sqrt_B = sqrtm(cov_matrix_next)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, cov_matrix_current), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A

    def transport_to_tangent(self, delta_matrix, tangent_plane_start, tangent_plane_end):
        inv_B = np.linalg.inv(tangent_plane_end)
        A_invB = np.dot(tangent_plane_start, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, delta_matrix), E.T)
        return transported_Delta

class CalculeTrajectoire:
    def __init__(self, cov_matrices, oublie):
        self.cov_matrices = cov_matrices
        self.trajectory = []
        self.transformation = Transformation()
        self.oublie = oublie

    def compute_trajectory(self):
        for i in range(len(self.cov_matrices) - 1):
            cov_matrix_current = self.cov_matrices[i]
            cov_matrix_next = self.cov_matrices[i + 1]
            # Projection de la matrice actuelle vers le plan tangent de la matrice suivante
            projected_matrix = self.transformation.project_to_tangent(cov_matrix_current, cov_matrix_next)
            self.trajectory.append(projected_matrix)
            new_trajectory = projected_matrix
            # Transport de toutes les projections actuelles vers le plan tangent de la matrice suivante
            for j in range(len(self.trajectory)-2, -1, -1):
                self.trajectory[j] = self.transformation.transport_to_tangent(self.trajectory[j], cov_matrix_current, cov_matrix_next) + new_trajectory
                # Additionner la matrice transportée à la matrice projetée
                new_trajectory += self.trajectory[j]
                
            # Limiter la taille de la trajectoire à p
            if len(self.trajectory) > self.oublie:
                self.trajectory.pop(0)

        return self.trajectory

# Exemple d'utilisation
# Générer des données initiales
generateur = GenerateurDonnees()
raw_data_initiale = generateur.generer_donnees()
donnees_initiales = raw_data_initiale.get_data()

# Calculer les matrices de covariance initiales
calcul_cov = CalculCovariance(p_points=256, stride=128)
matrices_covariance_initiales = calcul_cov.calculer_matrices_covariance(donnees_initiales)

pipeline = CalculeTrajectoire(matrices_covariance_initiales, oublie=5)
trajectory = pipeline.compute_trajectory()

print(trajectory)


Creating RawArray with float64 data, n_channels=3, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     9.996 secs
Ready.
[array([[-7.19075441e+16,  1.91694371e+17, -2.50286833e+16],
       [ 1.91694371e+17, -2.20937086e+16, -3.83941043e+16],
       [-2.50286833e+16, -3.83941043e+16,  6.23872071e+16]]), array([[-3.54380789e+16,  9.44724276e+16, -1.23348456e+16],
       [ 9.44724276e+16, -1.08884067e+16, -1.89217045e+16],
       [-1.23348456e+16, -1.89217045e+16,  3.07461866e+16]]), array([[-1.76908060e+16,  4.71609478e+16, -6.15759566e+15],
       [ 4.71609478e+16, -5.43552859e+15, -9.44577735e+15],
       [-6.15759566e+15, -9.44577735e+15,  1.53485979e+16]]), array([[-1.67191682e+16,  4.45707120e+16, -5.81940006e+15],
       [ 4.45707120e+16, -5.13699132e+15, -8.92698390e+15],
       [-5.81940006e+15, -8.92698390e+15,  1.45056019e+16]]), array([[-2.87690115e+17,  7.66937274e+17, -1.00135596e+17],
       [ 7.66937274e+17, -8.83932507e+16, -1.53608421e+17],
       [-1.00135596e+17, 